In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")

print("Dataset baixado em:", path)

Using Colab cache for faster access to the 'cbis-ddsm-breast-cancer-image-dataset' dataset.
Dataset baixado em: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset


In [12]:
import kagglehub
import os
import pandas as pd
import glob

# 1. Download do Dataset (baseado na sua imagem 1)
print("--- Baixando Dataset awsaf49/cbis-ddsm ---")
path = kagglehub.dataset_download("awsaf49/cbis-ddsm-breast-cancer-image-dataset")
print(f"Dataset baixado em: {path}")

# 2. Localizar os arquivos CSV (baseado na sua imagem 2: estão na pasta 'csv')
csv_folder = os.path.join(path, 'csv')
jpeg_folder = os.path.join(path, 'jpeg')

print(f"\nProcurando CSVs em: {csv_folder}")
mass_train_path = os.path.join(csv_folder, 'mass_case_description_train_set.csv')
mass_test_path = os.path.join(csv_folder, 'mass_case_description_test_set.csv')

# Vamos usar apenas Massas (Tumores) para este experimento, pois é o padrão para classificação geral
# Se quiser incluir Calcificações, basta repetir a lógica para 'calc_case_description...'
dfs = []
if os.path.exists(mass_train_path):
    dfs.append(pd.read_csv(mass_train_path))
if os.path.exists(mass_test_path):
    dfs.append(pd.read_csv(mass_test_path))

if not dfs:
    raise FileNotFoundError("ERRO CRÍTICO: Não foi possível encontrar os arquivos CSV na pasta 'csv'.")

df_full = pd.concat(dfs, ignore_index=True)
print(f"Total de registros de metadados carregados: {len(df_full)}")

# 3. Mapeamento de Classes (Regra do Artigo)
def get_label(pathology):
    if 'MALIGNANT' in pathology:
        return 'Malignant'
    elif 'BENIGN_WITHOUT_CALLBACK' in pathology:
        return 'Normal' # Simulando classe Normal
    elif 'BENIGN' in pathology:
        return 'Benign'
    return None

df_full['label'] = df_full['pathology'].apply(get_label)
df_full = df_full.dropna(subset=['label']) # Remove classes desconhecidas

# 4. Localizar as Imagens Físicas (.jpg)
# A pasta 'jpeg' geralmente contém subpastas. Vamos indexar tudo para busca rápida.
print("\nIndexando imagens na pasta 'jpeg' (aguarde)...")
image_map = {}
# Busca recursiva por todos os arquivos .jpg dentro da pasta jpeg
for img_path in glob.glob(os.path.join(jpeg_folder, '**', '*.jpg'), recursive=True):
    filename = os.path.basename(img_path)
    # O nome do arquivo geralmente contém o ID (ex: Mass-Training_P_00001...)
    # Vamos usar o nome do arquivo como chave para facilitar
    image_map[filename] = img_path

print(f"Total de imagens .jpg encontradas no disco: {len(image_map)}")

# 5. Cruzar Metadados (CSV) com Arquivos Reais (JPEG)
final_data = []

for idx, row in df_full.iterrows():
    # O CSV tem o caminho original "Mass-Training_P_00001_LEFT_CC/..."
    # Vamos tentar achar o arquivo correspondente no nosso mapa

    # Extrai o ID base da imagem (ex: Mass-Training_P_00001_LEFT_CC)
    base_id = row['image file path'].split('/')[0].strip()

    # Procura no mapa se existe algum arquivo que contenha esse ID
    # E preferencialmente que seja a imagem completa (FULL) ou cortada (CROP/ROI)
    # O artigo usa imagens processadas, vamos tentar pegar a ROI (corte do tumor) se disponível,
    # ou a imagem FULL se for a única opção.

    found_path = None

    # Tenta achar a ROI primeiro (imagens cortadas focadas no câncer são melhores para CNN)
    for fname, fpath in image_map.items():
        if base_id in fname and "CROP" in fname:
             found_path = fpath
             break

    # Se não achou crop, tenta FULL
    if not found_path:
        for fname, fpath in image_map.items():
            if base_id in fname and "FULL" in fname:
                 found_path = fpath
                 break

    if found_path:
        final_data.append({
            'filepath': found_path,
            'label': row['label']
        })

df_dataset = pd.DataFrame(final_data)
print(f"\nImagens vinculadas com sucesso: {len(df_dataset)}")
print(df_dataset['label'].value_counts())

# 6. Balanceamento e Split Final (Para o Artigo)
from sklearn.model_selection import train_test_split

# Tentar pegar 650 de cada classe (ou o máximo disponível se tiver menos)
balanced_dfs = []
for label in ['Normal', 'Benign', 'Malignant']:
    subset = df_dataset[df_dataset['label'] == label]
    if len(subset) >= 650:
        balanced_dfs.append(subset.sample(n=650, random_state=42))
    else:
        print(f"AVISO: Classe {label} tem apenas {len(subset)} imagens (Meta era 650).")
        balanced_dfs.append(subset)

if balanced_dfs:
    df_final_balanced = pd.concat(balanced_dfs)

    # Divisão 70% Treino / 30% Teste
    train, test = train_test_split(df_final_balanced, test_size=0.3, stratify=df_final_balanced['label'], random_state=42)

    train.to_csv('ddsm_train.csv', index=False)
    test.to_csv('ddsm_test.csv', index=False)

    print("\n--- SUCESSO! ---")
    print(f"Arquivo 'ddsm_train.csv' criado com {len(train)} imagens.")
    print(f"Arquivo 'ddsm_test.csv' criado com {len(test)} imagens.")
    print("Agora você pode prosseguir para o pré-processamento e treinamento.")
else:
    print("Falha ao criar o dataset balanceado.")

--- Baixando Dataset awsaf49/cbis-ddsm ---
Using Colab cache for faster access to the 'cbis-ddsm-breast-cancer-image-dataset' dataset.
Dataset baixado em: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset

Procurando CSVs em: /kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv
Total de registros de metadados carregados: 1696

Indexando imagens na pasta 'jpeg' (aguarde)...
Total de imagens .jpg encontradas no disco: 600

Imagens vinculadas com sucesso: 0


KeyError: 'label'